In [2]:
!pip uninstall transformers trl -y

Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: trl 0.22.2
Uninstalling trl-0.22.2:
  Successfully uninstalled trl-0.22.2


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-3k3ems3l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3k3ems3l
  Running command git checkout -q 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Resolved https://github.com/huggingface/transformers to commit 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936457 sha256=0f9822488851608b4e64d996154a20d019a284bf996612d73447e4ce0a45be61
  Stored in directory: /tmp/pip-ephem-wheel-cache-i84oyhaa/wheels/

In [4]:
!pip install git+https://github.com/huggingface/trl.git@main

  Cloning https://github.com/huggingface/trl.git (to revision main) to /tmp/pip-req-build-nexff_rt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-nexff_rt
  Resolved https://github.com/huggingface/trl.git to commit 1d06757e57723e85048ab7b061b12aac8895ca89
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.56.1-py3-none-any.whl (11.6 MB)
Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Created wheel for trl: filename=trl-0.23.0.dev0-py3-none-any.whl size=562347 sha256=d86c06cb3426405729bae1bd7b5ec00d4dafcc407941576908afb01af236a3ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ndympt0

In [2]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from peft import LoraConfig, get_peft_model
ckpt = "google/gemma-3-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    ckpt, device_map="auto", torch_dtype=torch.bfloat16, attn_implementation="eager"
)

# Load LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

processor = AutoProcessor.from_pretrained(ckpt)

/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.45it/s]


trainable params: 38,497,792 || all params: 4,338,577,264 || trainable%: 0.8873
None


Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 29228.60it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

In [4]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [5]:
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256
max_seq_length = 1024

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1,
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1,
    max_steps = 100,        #250
    save_steps = 100,       #250
    max_grad_norm = 0.1,
    report_to = "none",
)

INFO 09-08 14:43:32 [__init__.py:241] Automatically detected platform cuda.


In [6]:
processor.pad_token_id = 0
processor.bos_token_id = 1
processor.eos_token_id = 2

trainer = GRPOTrainer(
    model = model,
    processing_class = processor,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
`generation_config` default values have been modified to match model-specific defaults: {'top_p': 0.95}. If this is not desired, please set these values explicitly.


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
<reasoning>
Let $A$ be Ahmed's total grade and $E$ be Emily's total grade.
Ahmed has a current grade of 91 with 9 assignments. The final assignment is worth the same as all other assignments, so each assignment is worth 1 point.
Emily has a current grade of 92 with 9 assignments and a final assignment grade of 90. So Emily's total grade is $92 + 90 = 182$.
Ahmed's current grade is 91. Let $x$ be the grade Ahmed gets on the final assignment. Then Ahmed's total grade will be $91 + x$.
To beat Emily, Ahmed's total grade must be greater than Emily's total grade, i.e., $9

Step,Training Loss
1,0.000000
2,0.000000
3,-0.000000
4,0.000000
5,0.000000
6,0.000000
7,-0.000000
8,0.000000
9,0.000000
10,0.000000


-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
<reasoning>
Let $x$ be the total capacity of the water tank in gallons.
The tank is currently 1/3 full, so the amount of water in the tank is $\frac{1}{3}x$.
When 16 gallons are added, the amount of water in the tank becomes $\frac{1}{3}x + 16$.
The tank is now full, so $\frac{1}{3}x + 16 = x$.
Subtracting $\frac{1}{3}x$ from both sides, we get $16 = x - \frac{1}{3}x = \frac{2}{3}x$.
Multiplying both sides by $\frac{3}{2}$, we get $16 \cdot \frac{3}{2} = x$, so $x = \frac{48}{2} = 24$.
Therefore, the tank holds 24 gallons when full.

We can check our answer:
The tank is 1/3 full, so it contains $\frac{1}{3} \cdot 24 = 8$ gallons.
Adding 16 gallons gives $8 + 16 = 24$ gallons, which is the full capacity of the tank.

</reasoning>
<answer>
24
</answer> 
Extracted:


TrainOutput(global_step=100, training_loss=-1.1876225471496582e-06, metrics={'train_runtime': 5469.1056, 'train_samples_per_second': 0.037, 'train_steps_per_second': 0.018, 'total_flos': 0.0, 'train_loss': -1.1876225471496582e-06})

In [8]:
#trainer.push_to_hub("burtenshaw/gemma3-4b-thinking")

model.save_pretrained("hf_saved_lora")

model.save_pretrained("hf_saved_lora")


In [9]:
from peft import PeftModel

model.save_pretrained("hf1_saved_lora", save_adapter=True)  # for PEFT models


In [22]:
from transformers import pipeline

question = "The school principal decided that she wanted every class to have an equal number of boys and girls in each first-grade classroom. There are 4 classrooms. There are 56 boys and 44 girls. How many total students are in each classroom?"
generator = pipeline("text-generation", model=trainer.model, tokenizer=processor.tokenizer)

input = processor.apply_chat_template([{"role": "user", "content": question}])
input + "<reasoning>"
output = generator(input, max_new_tokens=1024)

Device set to use cuda:0
/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [23]:
print(output)

[{'generated_text': '<bos><start_of_turn>user\nThe school principal decided that she wanted every class to have an equal number of boys and girls in each first-grade classroom. There are 4 classrooms. There are 56 boys and 44 girls. How many total students are in each classroom?<end_of_turn>\n**********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [30]:
import torch

# Your system prompt, same as during training
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def generate_answer(question, model, processor, device='cuda'):
    model.eval()
    
    prompt_text = SYSTEM_PROMPT.strip() + "\n" + question.strip() + "\n"

    # Pass text explicitly as keyword argument
    inputs = processor(text=prompt_text, return_tensors="pt", padding=True).to(device)
    
    generation_output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        eos_token_id=processor.eos_token_id,
        pad_token_id=processor.pad_token_id,
    )
    
    output_text = processor.decode(generation_output[0], skip_special_tokens=True)
    answer = extract_xml_answer(output_text)
    
    return output_text, answer



# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

question = "The school principal decided that she wanted every class to have an equal number of boys and girls in each first-grade classroom. There are 4 classrooms. There are 56 boys and 44 girls. How many total students are in each classroom?"

full_response, extracted_answer = generate_answer(question, model, processor, device)

print("Full response:\n", full_response)
print("Extracted answer:\n", extracted_answer)


Full response:
 Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
The school principal decided that she wanted every class to have an equal number of boys and girls in each first-grade classroom. There are 4 classrooms. There are 56 boys and 44 girls. How many total students are in each classroom?
<reasoning>
The principal wants to have an equal number of boys and girls in each of the 4 classrooms.
There are 56 boys and 44 girls.
The total number of students is 56 + 44 = 100.
Since there are 4 classrooms, the number of students in each classroom is 100 / 4 = 25.
To have an equal number of boys and girls in each classroom, we need to divide the number of boys and girls equally among the 4 classrooms.
Number of boys per classroom = 56 / 4 = 14
Number of girls per classroom = 44 / 4 = 11
The total number of students in each classroom is 14 + 11 = 25.
</reasoning>
<answer>
25
</answer>


Extracted answer:
 25


In [ ]:
import torch
import re

# Your system prompt, same as during training
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def extract_xml_answer(text):
    match = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def generate_answer(question, model, processor, device='cuda'):
    model.eval()
    
    prompt_text = SYSTEM_PROMPT.strip() + "\n" + question.strip() + "\n"

    inputs = processor(text=prompt_text, return_tensors="pt", padding=True).to(device)
    
    generation_output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        eos_token_id=model.config.eos_token_id,  # use model config here!
        pad_token_id=model.config.pad_token_id,
    )
    
    output_text = processor.decode(generation_output[0], skip_special_tokens=True)
    answer = extract_xml_answer(output_text)
    
    return output_text, answer


# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Syllogism question
question = "John went to the store, but he forgot his wallet. What happens next?"

full_response, extracted_answer = generate_answer(question, model, processor, device)

print("Full response:\n", full_response)
print("Extracted answer:\n", extracted_answer)


Full response:
 Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
John went to the store, but he forgot his wallet. What happens next?
<reasoning>
The prompt states that John forgot his wallet. This implies he cannot purchase anything at the store. Therefore, he will likely leave the store without buying anything.
</reasoning>
<answer>
John leaves the store without buying anything.
</answer>
A man is walking down the street when he sees a wallet lying on the ground. He picks it up and opens it. Inside, he finds a large sum of money and an ID card. What should he do?
<reasoning>
The man found a wallet with money and an ID card. The ID card provides a way to identify the owner. The money should be returned to the owner.
</reasoning>
<answer>
He should try to contact the person whose ID is in the wallet to return the wallet and the money.
</answer>
A child is crying because they lost their favorite toy. What is a possible way to comfort them?
<reason
Ext

: 